#  SGmarkets ROTB strategy endpoint

**The [compute-strategy endpoint](https://analytics-api.sgmarkets.com/rotb/v1/swagger/ui/index#!/Swaption/Swaption_Strategy) enables to compute the historical performances of vol strategies and of its components.**  
**The calculation can be performed using either relative (analytics) or absolute (backtests) parameters.**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

import datetime as dt
import numpy as np
import pandas as pd

from pandas.tseries.offsets import BDay

from sgmarkets_api_auth import Api
from sgmarkets_plot import Plot
import sgmarkets_api_analytics_rotb as ROTB
from sgmarkets_api_auth.util import topickle, unpickle
import sgmarkets_api_analytics_rotb.biz as biz

from IPython.display import display, Markdown, Latex, Image

<IPython.core.display.Javascript object>

## 1 - Authentication

In [3]:
a = Api(verbose=True)

Variables in /Users/Olivier/my_secret.txt
SG_LOGIN=o**********************************m
SG_PASSWORD=m****************d
PROXY_LOGIN=m*******************n
PROXY_PASSWORD=m************d
PROXY_HOST=m************t
PROXY_PORT=9*******9
Token endpoint: https://login.sgmarkets.com/richclient/gettoken
proxies: {}
Access token {b102****************************532e} loaded from file /Users/Olivier/my_token.txt


## 2 - Build Request

In [4]:
ep = ROTB.endpoint.v1_compute_strategy
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=2)
rq.targetCurrency='EUR'
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'DV01'

rq.curve = 'JPY LIBOR 6M,EUR EURIBOR 6M'
rq.expiry = '3M,6M'
rq.tenor = '10Y,5Y'
rq.strike= '@0bp'
rq.type = 'receiver'
rq.settlement = 'cash'
rq.pricingStrategy = 'Auto'
rq.nominal = int(1e6)
rq.strategyWeight = '1,-1'

rq.expand()
rq.info()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 5 dates i.e. total 10 prices.


A RequestROTB object has the properties after the `expand()` method:
+ `url`: https://analytics-api.sgmarkets.com/rotb/v1/compute-strategy
+ `dic_input`: user input (dictionary)
+ `df_top`: parameters of the request not in a leg (dataframe)
+ `df_leg`: parameters of the request in a leg (dataframe)
    + A leg contains the following parameters: ['curve', 'expiry', 'tenor', 'strike', 'nominal', 'type', 'settlement', 'pricingStrategy', 'strategyWeight', 'customEntries']
    + Each param can have multiple comma separated values
+ `li_dic_api`: data ready to be sent to the API - built from user input by `expand()` (list of dictionaries)
    + This is a list because the request may be split in several chunks
    + Legs are constructed from user input and by filling missing values by repition if needed.
        

## 3 - Make request

In [5]:
res1 = rq.call_api(a)

calling API...
chunk 1/1 done in 1.04 s


## 4 - Response
The **ROTB.endpoint.v1_compute_strategy** endpoint returns 2 results:
+ Strategy
+ Components

In [6]:
res1.info()


A PostprocessROTB object from ComputeStrategy endpoint has the properties:
    
+ Contains two sub-objects:
    +   `.components` : object of class computestrategycomponent   
    +    `.strategy`: object of class computestrategyprice
+ Common properties:
    + `df_req`: request data (dataframe)
    + `df_res`: response data (dataframe)
    + `dic_req_param`: params in request, each param contains a list of all values taken (dictionary)
    + `dic_res_param`: params in response, each param contains a list of all values taken (dictionary)
    + `raw_data`: raw data in response under key 'componentSeries' (dictionary)
+ Additional properties for .components sub-object:    
    + `df_set`: request and response data combined (dataframe)

and the methods:
+ `save()` to save the data as `.csv` and `.xlsx` files
        

### 4.1 - Save to Disk

In [7]:
res1.save()

file dump/SG_Research_ROTBComponents_20180403_193650.csv saved
file dump/SG_Research_ROTBStrategy_res_20180403_193650.csv saved
file dump/SG_Research_ROTBStrategy_req_20180403_193650.csv saved


### 4.2 - Result as Dataframe

In [8]:
res1.strategy.df_res

,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,-488.618669,2739.982418,54.723559,2740.212440,-15.093457,195.104600,-0.002037,0
2018-03-27,-488.282772,2783.291749,53.831316,2783.528277,-15.326481,194.972534,-0.001987,1
2018-03-28,-488.255471,2822.838652,53.075301,2823.097839,-15.543550,194.962208,-0.002005,2
2018-03-29,-488.118105,2694.156758,55.530205,2694.399903,-14.841409,194.911020,-0.002040,3
2018-03-30,-487.984631,2624.457665,56.942327,2624.689463,-14.621733,193.790747,-0.002068,4


### 4.3 - Filter Components

In [9]:
# slice the request into a Multindex DataFrame

dic_req_fix ={'curve':'JPY LIBOR 6M'}
values=['volNormal','forwardRate']
sli = ep.slice(res1.components,x=None,y='date',z=None,dic_req_fix=dic_req_fix,value=values, y_pos='index',other_pos='column')
display(sli.df_pivot)

sli = ep.slice(res1.components,x='expiry',y='date',z=None,dic_req_fix=dic_req_fix,value=values, y_pos='index',other_pos='column')
display(sli.df_pivot)

sli = ep.slice(res1.components,x='expiry',y='date',z='tenor',dic_req_fix=dic_req_fix,value=values, y_pos='index',other_pos='column')
display(sli.df_pivot)

,forwardRate,volNormal
curve,JPY LIBOR 6M,JPY LIBOR 6M
date,,
2018-03-26,0.002958,0.001427
2018-03-27,0.003095,0.001449
2018-03-28,0.003108,0.001470
2018-03-29,0.003155,0.001405
2018-03-30,0.003209,0.001377


,forwardRate,volNormal
curve,JPY LIBOR 6M,JPY LIBOR 6M
expiry,3M,3M
date,,
2018-03-26,0.002958,0.001427
2018-03-27,0.003095,0.001449
2018-03-28,0.003108,0.001470
2018-03-29,0.003155,0.001405
2018-03-30,0.003209,0.001377


,forwardRate,volNormal
curve,JPY LIBOR 6M,JPY LIBOR 6M
expiry,3M,3M
tenor,10Y,10Y
date,,
2018-03-26,0.002958,0.001427
2018-03-27,0.003095,0.001449
2018-03-28,0.003108,0.001470
2018-03-29,0.003155,0.001405
2018-03-30,0.003209,0.001377


## 5 - Example #2
### Straddle

In [10]:
ep = ROTB.endpoint.v1_compute_strategy
rq = ep.request()
start=dt.date.today()-dt.timedelta(days=10)
end=dt.date.today()-dt.timedelta(days=2)
rq.targetCurrency='EUR'
rq.errorMode = 'Full'
rq.startDate = start.strftime('%Y-%m-%d')
rq.endDate = end.strftime('%Y-%m-%d')

rq.weighting = 'DV01'

rq.curve = 'EUR EURIBOR 6M'
rq.expiry = '3M'
rq.tenor = '10Y'
rq.strike= '@0bp'
rq.type = 'receiver,payer'
rq.settlement = 'cash'
rq.pricingStrategy = 'Auto'
rq.nominal = int(1e6)
rq.strategyWeight = '1'

rq.expand()
rq.info()

**<span style="color:red;">WARNING</span> -**improper combinaison of parameters resulting in an unclear strategy, by default missing parameters will be filled by repetition

**<span style="color:black;">INFO</span> -** Your request contains 2 prices over 5 dates i.e. total 10 prices.


A RequestROTB object has the properties after the `expand()` method:
+ `url`: https://analytics-api.sgmarkets.com/rotb/v1/compute-strategy
+ `dic_input`: user input (dictionary)
+ `df_top`: parameters of the request not in a leg (dataframe)
+ `df_leg`: parameters of the request in a leg (dataframe)
    + A leg contains the following parameters: ['curve', 'expiry', 'tenor', 'strike', 'nominal', 'type', 'settlement', 'pricingStrategy', 'strategyWeight', 'customEntries']
    + Each param can have multiple comma separated values
+ `li_dic_api`: data ready to be sent to the API - built from user input by `expand()` (list of dictionaries)
    + This is a list because the request may be split in several chunks
    + Legs are constructed from user input and by filling missing values by repition if needed.
        

In [11]:
res1 = rq.call_api(a)

calling API...
chunk 1/1 done in 0.94 s


In [12]:
res1.strategy.df_res

,delta,forwardValue,gamma,spotValue,theta,vega,volNormal,tag
date,,,,,,,,
2018-03-26,9.453089e-07,14248.222628,39.890718,14261.188354,-77.506458,378.670321,0.007532,0
2018-03-27,9.462721e-07,14144.286325,40.265788,14157.150327,-76.941034,379.056112,0.007470,1
2018-03-28,9.469999e-07,14319.673681,39.833846,14332.693578,-77.895074,379.347746,0.007556,2
2018-03-29,9.468211e-07,13978.502827,40.790789,13991.163105,-76.038930,379.276070,0.007378,3
2018-03-30,9.472560e-07,13970.302651,40.852635,13982.819085,-76.828676,377.382425,0.007410,4


In [13]:
# slice the request into a Multindex DataFrame

dic_req_fix ={}
values=['volNormal','forwardRate']
sli = ep.slice(res1.components,x='type',y='date',z=None,dic_req_fix=dic_req_fix,value=values, y_pos='index',other_pos='column')

sli.df_pivot

forwardRate           volNormal          
type             payer  receiver     payer  receiver
date                                                
2018-03-26    0.010534  0.010534  0.003766  0.003766
2018-03-27    0.010345  0.010345  0.003735  0.003735
2018-03-28    0.010199  0.010199  0.003778  0.003778
2018-03-29    0.010228  0.010228  0.003689  0.003689
2018-03-30    0.010141  0.010141  0.003705  0.003705